### Helper Classes

First we get all of our helper modules. The prepare_EMG module will prepare the EMG data for phoneme recognition. The prepare_outputs module will prepare our target labels and align them with our EMG data. The module 'prepare_data' will help us read data from CSV into a dataframe. Finally, 'vis' will help visualize EMG data in both time and frequency domains. 

In [ ]:
%load_ext autoreload
%autoreload 2

import prepare_EMG, prepare_outputs, prepare_data, vis

EMG_Prep = prepare_EMG.EMG_preparer()
Output_Prep = prepare_outputs.output_preparer()
Data_Prep = prepare_data.data_preparer()



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
          0.00      0.02      0.02      0.04      0.04      0.06      0.06  \
0    78.065625  3.629224  0.839573  3.174895  1.251957  2.208909  1.631892   
1    81.932813  0.103984  0.130759  0.309920  0.064390  0.066917  0.193844   
2    81.610547  0.000919  0.088216  0.155386  0.019537  0.417452  0.116381   
3    81.507422  0.102635  0.009653  0.043715  0.095052  0.314954  0.227756   
4    80.850000  0.155503  0.197546  0.229949  0.048328  0.000701  0.013273   
5    81.236719  0.112354  0.056141  0.004062  0.008119  0.054205  0.101333   
6    80.772656  0.272261  0.141355  0.000662  0.071712  0.215915  0.154497   
7    81.339844  0.121409  0.035416  0.103136  0.015159  0.099398  0.092555   
8    81.417187  0.109337  0.133520  0.164354  0.112874  0.137802  0.280535   
9    81.584766  0.035747  0.029262  0.073996  0.024613  0.107165  0.467060   
10   81.816797  0.082701  0.026714  0.067038  0.000022

/home/brian/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Labeling the Data

First, we need to visualize a few EMG voltage graphs to find some sections that most likely contain no subvocalization. Then, we'll need to find some regions that almost certainly do. These two classes of EMG readouts will serve to train an identifier to help us automatically label EMG windows with phonemes. The model used here will most likely be an SVC, inside "prepare_outputs". It will process each EMG window in order, and when it finds one that most likely contains subvocalization, it applies the next phoneme as that window's label. 

In [ ]:
data_1 = Data_Prep.load('Sat Mar  4 00:44:23 2017')
data_1[:3000]

data_no_sv = Data_Prep.sv_detection()
data_no_sv

In [ ]:
import pandas 

data_1 = pandas.read_csv('Sat Mar  4 00:44:23 2017')
data_2 = pandas.read_csv('Sat Mar  4 00:45:02 2017')
data_3 = pandas.read_csv('Sat Mar  4 00:45:47 2017')
data_4 = pandas.read_csv('Sat Mar  4 00:47:01 2017')
data_5 = pandas.read_csv('Sat Mar  4 00:47:36 2017')
no_sv = data_1[:3000].append(data_2[:2000]).append(data_3[:2000]).append(data_4[:2000]).append(data_5[:2000])

with_sv = data_1[5000:9000].append(data_2[6000:10000]).append(data_3[3000:9000])

windows = EMG_Prep.process(with_sv)


In [21]:
a = 4/2
b = 4//2

a,b

(2.0, 2)